In [41]:
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np 

In [2]:
path = '/kaggle/input/train-data/train_share.tsv'
train = pd.read_table(path)

In [3]:
train

,graph_1,graph_2,graph_3,feature_1,bki_1,graph_4,feature_2,graph_5,feature_3,graph_6,...,graph_22,application_2,bki_14,bki_15,feature_37,bki_16,bki_17,mb_15,feature_38,target
0,73.0,25.0,63.0,0.0,-3.307791,59.0,36.0,19544.0,0.0,91.0,...,0.0,0,44211.0,31.0,390.414429,0.571132,0.000,12527.0,0.0,0
1,64.0,37.0,68.0,0.0,-2.527839,106.0,36.0,1457.0,0.0,180.0,...,6.0,0,5600.0,3.0,663.702500,1.447663,0.100,10276.0,0.0,0
2,73.0,34.0,80.0,0.0,-2.985949,72.0,44.0,13071.0,0.0,151.0,...,1.0,0,250889.0,8.0,NaN,0.234987,11.240,4588.0,0.0,0
3,170.0,99.0,203.0,0.0,-3.392264,228.0,NaN,7000.0,NaN,388.0,...,2.0,0,NaN,NaN,471.623097,NaN,NaN,22968.0,0.0,0
4,65.0,20.0,77.0,0.0,-2.657559,115.0,18.0,21725.0,0.0,190.0,...,13.0,1,0.0,0.0,411.162400,0.000000,29.402,15687.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1265169,2.0,2.0,1.0,0.0,-2.806880,5.0,NaN,0.0,NaN,7.0,...,1.0,0,NaN,NaN,297.153600,NaN,NaN,2658.0,0.0,0
1265170,15.0,6.0,12.0,0.0,-2.971577,14.0,15.0,0.0,0.0,26.0,...,2.0,0,NaN,NaN,56.500000,NaN,NaN,4694.0,0.0,0
1265171,240.0,135.0,356.0,0.0,-3.313476,401.0,35.0,689921.0,0.0,668.0,...,3.0,0,38000.0,4.0,552.291581,0.114097,25.567,183997.0,205.0,0
1265172,17.0,2.0,6.0,0.0,-3.871151,22.0,NaN,500.0,NaN,40.0,...,4.0,0,NaN,NaN,1475.000000,NaN,NaN,21792.0,0.0,0


In [4]:
for column in train.columns:
    print(f'{column}: {train[column].nunique(), train[column].isna().sum()}')

graph_1: (1788, 1554)
graph_2: (1238, 1554)
graph_3: (2333, 1554)
feature_1: (2, 21961)
bki_1: (1257152, 7826)
graph_4: (2279, 1554)
feature_2: (384, 237091)
graph_5: (310041, 1554)
feature_3: (11, 209540)
graph_6: (3643, 1554)
graph_7: (86, 1554)
graph_8: (82975, 3467)
graph_9: (1227, 1554)
graph_10: (1248921, 5908)
feature_4: (59344, 821634)
graph_11: (163816, 1554)
graph_12: (595, 1554)
graph_13: (1249539, 14785)
bki_2: (792604, 448337)
graph_14: (151445, 1554)
feature_5: (10, 231544)
mb_1: (43241, 63474)
graph_15: (352103, 3505)
mb_2: (380, 63474)
graph_16: (343450, 1554)
bki_3: (3325, 469819)
feature_6: (2, 0)
feature_7: (7, 231544)
mb_3: (3816, 63474)
feature_8: (15, 231544)
feature_9: (8, 209540)
graph_17: (3158, 1554)
bki_4: (1246964, 17924)
feature_10: (116, 231544)
feature_11: (6, 231544)
feature_12: (2, 21961)
feature_13: (10, 246097)
feature_14: (2, 21961)
mb_4: (380, 63474)
feature_15: (925412, 286710)
mb_5: (795, 63474)
graph_18: (1121458, 9620)
bki_5: (727289, 448337)
fe

In [5]:
categorical = ["feature_1", "feature_3", "feature_5", "feature_6", "feature_8", "feature_9", "feature_10", "feature_11", "feature_12", "feature_13", "feature_14", "feature_16", "feature_18", "feature_23", "feature_24", "feature_26", "feature_28", "graph_20", "graph_22", "application_2", "bki_15"]

In [14]:
def data_preprocessing(data):
    for column in categorical:
        data[column] = data[column].fillna(-1)
        data[column] = data[column].astype('str')
    data = data.fillna(0)
    return data

In [8]:
from sklearn.model_selection import train_test_split

train = data_preprocessing(train)
data_train, data_val  = train_test_split(train, test_size=0.2, random_state=42, shuffle=True)

y_train = data_train.copy()['target']
X_train = data_train.copy().drop(columns = 'target')
y_val = data_val.copy()['target']
X_val = data_val.copy().drop(columns = 'target')

model = CatBoostClassifier(
    loss_function = 'Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    logging_level='Silent',
    iterations=1000,
    task_type="GPU",
    devices='0',
    use_best_model=True
)

model.fit(
    X_train, y_train,
    cat_features=categorical,
    eval_set=(X_val, y_val),
    logging_level='Verbose'
)

Learning rate set to 0.039544
0:	learn: 0.9233929	test: 0.9239591	best: 0.9239591 (0)	total: 7.78s	remaining: 2h 9m 33s
1:	learn: 0.9234216	test: 0.9239354	best: 0.9239591 (0)	total: 7.94s	remaining: 1h 6m 3s
2:	learn: 0.9235006	test: 0.9239710	best: 0.9239710 (2)	total: 8.1s	remaining: 44m 52s
3:	learn: 0.9235510	test: 0.9240066	best: 0.9240066 (3)	total: 8.23s	remaining: 34m 10s
4:	learn: 0.9234819	test: 0.9240026	best: 0.9240066 (3)	total: 8.39s	remaining: 27m 49s
5:	learn: 0.9234917	test: 0.9240658	best: 0.9240658 (5)	total: 8.55s	remaining: 23m 36s
6:	learn: 0.9234700	test: 0.9240263	best: 0.9240658 (5)	total: 8.71s	remaining: 20m 35s
7:	learn: 0.9234660	test: 0.9240105	best: 0.9240658 (5)	total: 8.86s	remaining: 18m 18s
8:	learn: 0.9234986	test: 0.9240105	best: 0.9240658 (5)	total: 9.01s	remaining: 16m 32s
9:	learn: 0.9234986	test: 0.9240382	best: 0.9240658 (5)	total: 9.14s	remaining: 15m 5s
10:	learn: 0.9235194	test: 0.9240026	best: 0.9240658 (5)	total: 9.25s	remaining: 13m 52s


In [57]:
path = '/kaggle/input/test-data-allcups-f/test_share.tsv'
test = pd.read_table(path)
test = data_preprocessing(test).drop(columns = 'Unnamed: 0')
test

,graph_1,graph_2,graph_3,feature_1,bki_1,graph_4,feature_2,graph_5,feature_3,graph_6,...,feature_36,graph_22,application_2,bki_14,bki_15,feature_37,bki_16,bki_17,mb_15,feature_38
0,23.0,29.0,88.0,0.0,-1.976491,64.0,22.0,136633.46,0.0,181.0,...,968.63,4.0,0,0.0,0.0,161.438333,0.969675,30.495,8504.0,0.0
1,95.0,70.0,147.0,0.0,-3.282326,188.0,34.0,268502.00,0.0,305.0,...,243878.17,4.0,0,0.0,0.0,677.439361,0.555632,0.000,109387.0,0.0
2,92.0,52.0,130.0,0.0,-3.815265,135.0,34.0,205182.00,0.0,224.0,...,581071.10,1.0,0,500000.0,3.0,994.984760,0.324038,9.578,175186.0,23.0
3,31.0,8.0,53.0,0.0,-1.658297,81.0,31.0,17315.00,0.0,132.0,...,12833.17,2.0,0,68905.0,8.0,237.651296,0.539998,0.000,33381.0,0.0
4,129.0,69.0,204.0,0.0,-3.543214,271.0,38.0,194450.00,0.0,396.0,...,9334.91,40.0,2,3274000.0,9.0,98.262211,0.168971,8.983,14512.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301484,33.0,8.0,40.0,0.0,-2.709187,75.0,0.0,0.00,0.0,92.0,...,57633.49,3.0,0,0.0,-1.0,243.179283,0.000000,0.000,93819.0,0.0
301485,16.0,7.0,12.0,0.0,-2.714078,11.0,60.0,0.00,0.0,31.0,...,0.00,3.0,1,243000.0,10.0,0.000000,0.198806,0.000,12451.0,0.0
301486,219.0,90.0,267.0,0.0,-3.679674,288.0,36.0,53270.00,0.0,435.0,...,0.00,5.0,0,0.0,-1.0,0.000000,0.000000,0.000,16620.0,0.0
301487,46.0,19.0,51.0,0.0,-3.333991,44.0,55.0,5033.00,0.0,83.0,...,528157.43,0.0,0,0.0,-1.0,1956.138630,0.000000,0.000,19779.0,56.0


In [58]:
predict = model.predict_proba(test)[:,1:].flatten()

In [65]:
submission = pd.DataFrame({'id' : test.index, 'target' : predict})

In [66]:
submission

,id,target
0,0,0.060374
1,1,0.132942
2,2,0.101031
3,3,0.076739
4,4,0.127302
...,...,...
301484,301484,0.029278
301485,301485,0.026173
301486,301486,0.083727
301487,301487,0.018037


In [67]:
submission.to_csv('submission_F.csv', index=False)